In [233]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk # natural language tool kit

In [234]:
initialFrame = pd.read_csv('train.tsv', delimiter = '\t');
initialTestFrame = pd.read_csv('test 2.tsv', delimiter = '\t');

In [235]:
initialFrame

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
...,...,...,...,...
156055,156056,8544,Hearst 's,2
156056,156057,8544,forced avuncular chortles,1
156057,156058,8544,avuncular chortles,3
156058,156059,8544,avuncular,2


In [236]:
initialTestFrame

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine
...,...,...,...
66287,222348,11855,"A long-winded , predictable scenario ."
66288,222349,11855,"A long-winded , predictable scenario"
66289,222350,11855,"A long-winded ,"
66290,222351,11855,A long-winded


In [237]:
# Cleans the initial frame
def cleanInitialFrame(df):
    cleanList = [] # list to grow
    currentSentence = 0 # tracks current sentence
    sentenceIDs = {0} 
    # Iterate row by row
    for index, row in df.iterrows():
        # If it's the first element, add to list
        if (row['SentenceId'] == currentSentence):
            continue
        else:
            cleanList.append([row['PhraseId'], row['SentenceId'], row['Phrase'], row['Sentiment']]);
            currentSentence = row['SentenceId']
    
    # Return a clean frame
    return pd.DataFrame(cleanList, columns = ['PhraseId', 'SentenceId', 'Phrase', 'Sentiment'])

In [238]:
# Cleans the initial frame
def cleanInitialTestFrame(df):
    cleanList = [] # list to grow
    currentSentence = 0 # tracks current sentence
    sentenceIDs = {0} 
    # Iterate row by row
    for index, row in df.iterrows():
        # If it's the first element, add to list
        if (row['SentenceId'] == currentSentence):
            continue
        else:
            cleanList.append([row['PhraseId'], row['SentenceId'], row['Phrase']]);
            currentSentence = row['SentenceId']
    
    # Return a clean frame
    return pd.DataFrame(cleanList, columns = ['PhraseId', 'SentenceId', 'Phrase'])

In [239]:
df = cleanInitialFrame(initialFrame)
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,64,2,"This quiet , introspective and entertaining in...",4
2,82,3,"Even fans of Ismail Merchant 's work , I suspe...",1
3,117,4,A positively thrilling combination of ethnogra...,3
4,157,5,Aggressive self-glorification and a manipulati...,1


In [240]:
testdf = cleanInitialTestFrame(initialTestFrame)
testdf.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156076,8546,Kidman is really the only thing that 's worth ...
2,156154,8547,Once you get into its rhythm ... the movie bec...
3,156178,8548,I kept wishing I was watching a documentary ab...
4,156219,8549,"Kinnear does n't aim for our sympathy , but ra..."


In [241]:
def lowerAllPhrases(df):
    phrases_list = list(df['Phrase'])

    for i in range(len(phrases_list)):
        phrases_list[i] = phrases_list[i].lower()
    count = 0;
    for index, row in df.iterrows():
        df.at[index,'Phrase'] = phrases_list[count]
        count += 1

    return df

In [242]:
lowerAllPhrases(df)
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,a series of escapades demonstrating the adage ...,1
1,64,2,"this quiet , introspective and entertaining in...",4
2,82,3,"even fans of ismail merchant 's work , i suspe...",1
3,117,4,a positively thrilling combination of ethnogra...,3
4,157,5,aggressive self-glorification and a manipulati...,1


In [243]:
lowerAllPhrases(testdf)
testdf.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,an intermittently pleasing but mostly routine ...
1,156076,8546,kidman is really the only thing that 's worth ...
2,156154,8547,once you get into its rhythm ... the movie bec...
3,156178,8548,i kept wishing i was watching a documentary ab...
4,156219,8549,"kinnear does n't aim for our sympathy , but ra..."


In [244]:
# Remove non-ascii characters using str.replace()
def asciiClean(df):
    # iterate row by row
    for index, row in df.iterrows():
        old_str = row['Phrase']
        new_str = (old_str.encode('ascii','ignore')).decode()
        df.at[index, 'Phrase'] = new_str

In [245]:
asciiClean(df)
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,a series of escapades demonstrating the adage ...,1
1,64,2,"this quiet , introspective and entertaining in...",4
2,82,3,"even fans of ismail merchant 's work , i suspe...",1
3,117,4,a positively thrilling combination of ethnogra...,3
4,157,5,aggressive self-glorification and a manipulati...,1


In [246]:
asciiClean(testdf)
testdf.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,an intermittently pleasing but mostly routine ...
1,156076,8546,kidman is really the only thing that 's worth ...
2,156154,8547,once you get into its rhythm ... the movie bec...
3,156178,8548,i kept wishing i was watching a documentary ab...
4,156219,8549,"kinnear does n't aim for our sympathy , but ra..."


In [247]:
df[0:10]

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,a series of escapades demonstrating the adage ...,1
1,64,2,"this quiet , introspective and entertaining in...",4
2,82,3,"even fans of ismail merchant 's work , i suspe...",1
3,117,4,a positively thrilling combination of ethnogra...,3
4,157,5,aggressive self-glorification and a manipulati...,1
5,167,6,a comedy-drama of nearly epic proportions root...,4
6,199,7,"narratively , trouble every day is a plodding ...",1
7,214,8,"the importance of being earnest , so thick wit...",3
8,248,9,but it does n't leave you with much .,1
9,260,10,you could hate it for the same reason .,1


In [248]:
import re
def removeSpaces(df):
    for index, row in df.iterrows():
        # df['Phrase'] = df['Phrase'].replace([row['Phrase']], re.sub(r'\s+\'', "'", row['Phrase']))
        df.at[index,'Phrase'] = re.sub(r'\s+\'', "'", row['Phrase'])

In [249]:
removeSpaces(df)

In [250]:
removeSpaces(testdf)

In [251]:
df[0:10]

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,a series of escapades demonstrating the adage ...,1
1,64,2,"this quiet , introspective and entertaining in...",4
2,82,3,"even fans of ismail merchant's work , i suspec...",1
3,117,4,a positively thrilling combination of ethnogra...,3
4,157,5,aggressive self-glorification and a manipulati...,1
5,167,6,a comedy-drama of nearly epic proportions root...,4
6,199,7,"narratively , trouble every day is a plodding ...",1
7,214,8,"the importance of being earnest , so thick wit...",3
8,248,9,but it does n't leave you with much .,1
9,260,10,you could hate it for the same reason .,1


In [252]:
import contractions
contractions.add('n\'t', 'not')
def expandContractions(df):
    for index, row in df.iterrows():
        phrase = []
        for i in row['Phrase'].split():
            phrase.append(contractions.fix(i))
        string_version = ' '.join(phrase)
        df.at[index, 'Phrase'] = string_version            

In [253]:
expandContractions(df)

In [254]:
expandContractions(testdf)

In [255]:
import string
def removePunctuation(df):
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    for index,row in df.iterrows():
        phrase = []
        for i in row['Phrase'].split():
            phrase.append(regex.sub('', i))
        string_version = ' '.join(phrase)
        df.at[index, 'Phrase'] = string_version

In [256]:
removePunctuation(df)

In [257]:
removePunctuation(testdf)

In [258]:
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\andyv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\andyv\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [259]:
def removeStopWords(df):
    
    phrases_list = list(df['Phrase'])
    stop_words = set(stopwords.words('english'))

    for i in range(len(phrases_list)):
        word_tokens = word_tokenize(phrases_list[i])
        filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
        filtered_sentence = []
        for w in word_tokens:
            if w not in stop_words:
                filtered_sentence.append(w)
        phrases_list[i] = filtered_sentence
    
    for i in range(len(phrases_list)):
        phrases_list[i] = TreebankWordDetokenizer().detokenize(phrases_list[i])
    
    count = 0
    for index, row in df.iterrows():
        # df['Phrase'] = df['Phrase'].replace([row['Phrase']], phrases_list[count])
        df.at[index, 'Phrase'] = phrases_list[count]
        count += 1

    return df      

In [260]:
df.head(10)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,a series of escapades demonstrating the adage ...,1
1,64,2,this quiet introspective and entertaining ind...,4
2,82,3,even fans of ismail merchants work i suspect ...,1
3,117,4,a positively thrilling combination of ethnogra...,3
4,157,5,aggressive selfglorification and a manipulativ...,1
5,167,6,a comedydrama of nearly epic proportions roote...,4
6,199,7,narratively trouble every day is a plodding m...,1
7,214,8,the importance of being earnest so thick with...,3
8,248,9,but it does not leave you with much,1
9,260,10,you could hate it for the same reason,1


In [261]:
df = removeStopWords(df)
df.head(10)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,series escapades demonstrating adage good goos...,1
1,64,2,quiet introspective entertaining independent w...,4
2,82,3,even fans ismail merchants work suspect would ...,1
3,117,4,positively thrilling combination ethnography i...,3
4,157,5,aggressive selfglorification manipulative whit...,1
5,167,6,comedydrama nearly epic proportions rooted sin...,4
6,199,7,narratively trouble every day plodding mess,1
7,214,8,importance earnest thick wit plays like readin...,3
8,248,9,leave much,1
9,260,10,could hate reason,1


In [262]:
testdf = removeStopWords(testdf)
testdf.head(10)

,PhraseId,SentenceId,Phrase
0,156061,8545,intermittently pleasing mostly routine effort
1,156076,8546,kidman really thing worth watching birthday gi...
2,156154,8547,get rhythm movie becomes heady experience
3,156178,8548,kept wishing watching documentary wartime nava...
4,156219,8549,kinnear aim sympathy rather delivers performan...
5,156250,8550,ends well sort frenzied comic moments never click
6,156272,8551,hoot half great way american people see candid...
7,156324,8552,weight piece unerring professionalism chilly p...
8,156362,8553,film contains good jokes good scenes barely mo...
9,156405,8554,offbeat sometimes gross surprisingly appealing...


In [263]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import wordnet

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\andyv\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\andyv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\andyv\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [264]:
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize     
def tag_words(df):
    allPhrases = list(df['Phrase'])
    for index,row in df.iterrows():
        tokenized = word_tokenize(allPhrases[index])
        tagged = nltk.pos_tag(tokenized)
        df.at[index, 'Phrase'] = tagged

def determine_root(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def lemmatize_phrases(df):
    lemmatizer = WordNetLemmatizer()
    allPhrases = list(df['Phrase'])
    for index,row in df.iterrows():
        final_sentence = []
        final_tag = list(map(lambda x: (x[0], determine_root(x[1])), allPhrases[index]))
        for word, tag in final_tag:
            if tag is None:
                final_sentence.append(word)
            else:
                final_sentence.append(lemmatizer.lemmatize(word, tag))
        final_sentence = ' '.join(final_sentence)
        df.at[index, 'Phrase'] = final_sentence             

In [265]:
tag_words(df)
lemmatize_phrases(df)

In [266]:
tag_words(testdf)
lemmatize_phrases(testdf)

In [267]:
df.head(10)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,series escapades demonstrating adage good goos...,1
1,64,2,quiet introspective entertaining independent w...,4
2,82,3,even fan ismail merchant work suspect would ha...,1
3,117,4,positively thrill combination ethnography intr...,3
4,157,5,aggressive selfglorification manipulative whit...,1
5,167,6,comedydrama nearly epic proportion root sincer...,4
6,199,7,narratively trouble every day plod mess,1
7,214,8,importance earnest thick wit play like read ba...,3
8,248,9,leave much,1
9,260,10,could hate reason,1


In [268]:
# TO DO:
# * reduce to multi-dimensional vector
#     - We have options including: Bag of words
#     - BERT
#     - TF-IDF
# * classification
#     - try a bunch of classifiers
# * graphing
#     - graph the outputs of our classifiers
# * presentation
#     - create a presentation

In [278]:
class Selene(object):
    
    def __init__(self, df, testdf):
        
        self.df = df
        self.testdf = testdf
        self.wordsDict = {}
        self.wordSentOcc = {}
        self.wordSentVal = {}
    
    def train(self, printer = False):
        # Tokenizes phrases and finds frequency & sentiment
        self.setUniqueWords()
        self.setFrequencies()
        self.findSentFrequency()
        if printer:
            counter = 0;
            print("Words = [Frequency, 0 Freq, 1 Freq, 2 Freq, 3 Freq, 4 Freq, 5 Freq]")
            for key, value in self.wordsDict.items():
                print("Word #", counter, ":", key, "-", value)
                counter += 1
    
    def test(self, weight = 0.1, minFreq = 3):
        self.getSentiment(weight, minFreq)
    
    # Finds unique words in the list of phrases and puts into list
    def setUniqueWords(self):
        
        phrases = list(self.df['Phrase'])
        
        for index, row in df.iterrows():
            sentence = row['Phrase']
            for w in word_tokenize(sentence): # word_tokens:
                if w.isalpha():
                    self.wordsDict[w] = [0,0,0,0,0,0]
                    
    def setFrequencies(self):
        
        for index, row in df.iterrows():
            sentence = row['Phrase']
            for word in sentence.split():
                if word in self.wordsDict:
                    self.wordsDict[word][0] += 1           
    
    # Finds frequencies of the sentiment per word
    def findSentFrequency(self):
                    
        for index, row in df.iterrows():
            sentiment = int(row['Sentiment'])
            for word in row['Phrase'].split():
                if word in self.wordsDict:
                    try:
                        self.wordsDict[word][sentiment + 1] += 1
                    except:
                        print(sentiment + 1)
            
    # Finds probabilities for a word's sentiment
    def findProbabilities(self, key):
        
        prob0 = self.wordSentOcc[key][0 + 1] / self.wordsDict[key]
        prob1 = self.wordSentOcc[key][1 + 1] / self.wordsDict[key]
        prob2 = self.wordSentOcc[key][2 + 1] / self.wordsDict[key]
        prob3 = self.wordSentOcc[key][3 + 1] / self.wordsDict[key]
        prob4 = self.wordSentOcc[key][4 + 1] / self.wordsDict[key]
        return prob0, prob1, prob2, prob3, prob
    
    # Finds the average sentiment of a word
    def findWordSentVal(self):
        
        for key, value in self.wordSentOcc.items():
            sentVal = (((value[0 + 1] * 0) + (value[1 + 1] * 1) + (value[2 + 1] * 2)
                    + (value[3 + 1] * 3) + (value[4 + 1] * 4)) / self.wordsDict[key])
            prob0, prob1, prob2, prob3, prob4 = self.findProbabilities(key)
            self.wordSentVal[key] = {'Avg Value': sentVal,'P0': prob0,
                                     'P1': prob1, 'P2': prob2, 
                                     'P3': prob3, 'P4': prob4}
            

    
    # Gets probability weights
    def getWeight(self, word, weightPercent, minimumOccurances):
        
        mystery = np.random.rand(0, 1)
        if self.wordsDict[word] >= minimumOccurances:
            if (mystery >= 0 and mystery <= self.wordSentVal[word]['P0']):
                return -2 * weightPercent
            if (mystery >= self.wordSentVal[word]['P0'] and mystery <= self.wordSentVal[word]['P0'] + self.wordSentVal[word]['P1']):
                return -1 * weightPercent
            if (mystery >= self.wordSentVal[word]['P0'] + self.wordSentVal[word]['P1'] and mystery <= 
                self.wordSentVal[word]['P1'] + self.wordSentVal[word]['P2']):
                return 0
            if (mystery >= self.wordSentVal[word]['P1'] + self.wordSentVal[word]['P2'] and mystery <= 
                self.wordSentVal[word]['P2'] + self.wordSentVal[word]['P3']):
                return 1 * weightPercent
            if (mystery >= self.wordSentVal[word]['P2'] + self.wordSentVal[word]['P3'] and mystery <= 
                self.wordSentVal[word]['P3'] + self.wordSentVal[word]['P4']):
                return 2 * weightPercent
        if self.wordsDict[word] < minimumOccurances:
            if (mystery >= 0 and mystery <= self.wordSentVal[word]['P0']):
                return -2 * weightPercent
            if (mystery >= self.wordSentVal[word]['P0'] and mystery <= self.wordSentVal[word]['P0'] + self.wordSentVal[word]['P1']):
                return -1 * weightPercent
            if (mystery >= self.wordSentVal[word]['P0'] + self.wordSentVal[word]['P1'] and mystery <= 
                self.wordSentVal[word]['P1'] + self.wordSentVal[word]['P2']):
                return 0
            if (mystery >= self.wordSentVal[word]['P1'] + self.wordSentVal[word]['P2'] and mystery <= 
                self.wordSentVal[word]['P2'] + self.wordSentVal[word]['P3']):
                return 1 * weightPercent
            if (mystery >= self.wordSentVal[word]['P2'] + self.wordSentVal[word]['P3'] and mystery <= 
                self.wordSentVal[word]['P3'] + self.wordSentVal[word]['P4']):
                return 2 * weightPercent 
    
    # Gets the sentiment of a phrase
    def getSentiment(self, weightPercent = .1, minimumOccurances = 3):
        
        Sentiments = []
        
        for index, row in self.testdf.iterrows():
            phraseVal = 0
            for word in row['Phrase'].split():
                if word in self.wordSentVal:
                    weight = self.getWeight(word, weightPercent, minimumOccurances)
                    phraseVal += self.wordSentVal[word]['Avg Value'] + weight
                print(word)
                # Neutral sentiment for unknown word
                # else word not in self.wordSentVal.keys():
                #     phraseVal += 2
            if len(row['Phrase'].split()) > 0:
                sentVal = phraseVal/len(row['Phrase'].split())
                Sentiments.append(sentVal)
            else:
                Sentiments.append(0)
                        
        # testdf['Sentiment'] = Sentiments
        for value in Sentiments:
            # print(value)
            continue
            
        print(Sentiments[1], Sentiments[5])
        return Sentiments
                
                
        

In [279]:
selene = Selene(df, testdf)

In [280]:
selene.train()

In [281]:
sent = selene.test()

intermittently
please
mostly
routine
effort
kidman
really
thing
worth
watch
birthday
girl
film
stagetrained
jez
butterworth
lrb
mojo
rrb
serve
yet
another
example
sad
decline
british
comedy
postfull
monty
world
get
rhythm
movie
become
heady
experience
keep
wish
watch
documentary
wartime
navajos
accomplish
instead
specious
hollywood
hooha
kinnear
aim
sympathy
rather
delivers
performance
strike
skill
depth
end
well
sort
frenzied
comic
moment
never
click
hoot
half
great
way
american
people
see
candidate
like
give
15cent
stump
speech
weight
piece
unerring
professionalism
chilly
production
fascination
embed
lurid
topic
prove
recommendation
enough
film
contains
good
joke
good
scene
barely
moment
carveys
saturday
night
livehoned
mimicry
rise
level
embarrassment
offbeat
sometimes
gross
surprisingly
appeal
animate
film
true
meaning
holiday
suggests
wideranging
effect
medium
manipulation
kind
report
do
supposedly
liberal
medium
intimate
ultimately
tragic
heartache
maverick
individual
like
hatfie

In [273]:
print(sent)

None
